# 🚀 Notebook 11: End-to-End Pipeline Export

**Autor:** Gian  
**Fecha:** 2026-01-19  
**Objetivo:** Consolidar todos los pasos de preprocesamiento, ingeniería de características y el modelo entrenado en un único objeto `Pipeline` de scikit-learn exportable. Esto garantiza que el modelo pueda ser desplegado y usado para predicciones con datos crudos nuevos.

---

## 📋 Contenido

1. Configuración del entorno
2. Carga de Datos (Crudos y Modelo)
3. Definición de Transformadores Personalizados (Feature Engineering)
4. Construcción del Pipeline
5. Validación del Pipeline (Sanity Check)
6. Entrenamiento Final del Preprocesador
7. Exportación del Pipeline Completo

---
## 1. Configuración del Entorno

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import joblib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


---
## 2. Carga de Datos y Modelo
Necesitamos:
1.  `data_clean.csv`: Los datos "crudos" originales para ajustar los scalers.
2.  `09_best_model.pkl`: El modelo ya entrenado y optimizado.

In [2]:
# Rutas
OUTPUT_PATH = Path("../../outputs/gian")
RAW_DATA_PATH = OUTPUT_PATH / "data" / "data_clean.csv"
MODEL_PATH = OUTPUT_PATH / "models" / "09_best_model.pkl"
PIPELINE_PATH = OUTPUT_PATH / "pipelines"
PIPELINE_PATH.mkdir(parents=True, exist_ok=True)

# Cargar Datos Crudos
df_raw = pd.read_csv(RAW_DATA_PATH)
X_raw = df_raw.drop(columns=['cancelacion', 'cliente_id'], errors='ignore')
y_raw = df_raw['cancelacion']

# Cargar Modelo Entrenado
trained_model = joblib.load(MODEL_PATH)

print(f"✅ Datos crudos cargados: {X_raw.shape}")
print(f"✅ Modelo cargado: {type(trained_model).__name__}")

✅ Datos crudos cargados: (9701, 65)
✅ Modelo cargado: RandomForestClassifier


---
## 3. Definición de Transformadores Personalizados
Creamos clases de scikit-learn para replicar la lógica de Feature Engineering exactamente como se hizo en el Notebook 06.

In [3]:
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self, ref_date=None):
        # Si no se pasa fecha, se usará el momento de transformación (para producción)
        # O se puede fijar durante el fit para consistencia si es necesario.
        self.ref_date = ref_date
        pass

    def fit(self, X, y=None):
        # Calcular la fecha de referencia máxima del conjunto de entrenamiento si no se provee
        if self.ref_date is None:
             if 'ultimo_contacto_soporte' in X.columns:
                # Asumimos que X es un DataFrame
                # Convertir temp para sacar max sin modificar X in-place
                temp_dates = pd.to_datetime(X['ultimo_contacto_soporte'], errors='coerce')
                self.ref_date_ = temp_dates.max()
        else:
            self.ref_date_ = pd.to_datetime(self.ref_date)
        return self

    def transform(self, X):
        # Evitar modificar el original
        X_out = X.copy()
        
        # 1. Intensidad de Uso
        if 'conexiones_mensuales' in X_out.columns and 'dias_activos_semanales' in X_out.columns:
            X_out['intensidad_uso'] = X_out['conexiones_mensuales'] / (X_out['dias_activos_semanales'] * 4 + 1)
            
        # 2. Ratio Carga Financiera
        if 'cargo_mensual' in X_out.columns and 'ingresos_totales' in X_out.columns:
            X_out['ratio_carga_financiera'] = X_out['cargo_mensual'] / (X_out['ingresos_totales'] + 1)
            
        # 3. Dias Desde Ultimo Contacto
        if 'ultimo_contacto_soporte' in X_out.columns:
            # Convertir a datetime
            dates = pd.to_datetime(X_out['ultimo_contacto_soporte'], errors='coerce')
            
            # Usar fecha de referencia aprendida en fit (o la actual si se prefiere lógica dinámica)
            # Para reproducir el entrenamiento, usamos la ref_date del training set.
            
            if hasattr(self, 'ref_date_'):
                anchor = self.ref_date_
            else:
                anchor = dates.max() # Fallback
                
            X_out['dias_desde_ultimo_contacto'] = (anchor - dates).dt.days
            X_out['dias_desde_ultimo_contacto'] = X_out['dias_desde_ultimo_contacto'].fillna(-1)
            
        return X_out

class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Seleccionar solo las columnas necesarias y en el orden correcto
        # Verificar que existan, si no, dar error o warning (aquí asumimos happy path)
        return X[self.columns]

print("✅ Clases FeatureEngineer y ColumnSelector definidas.")

✅ Clases FeatureEngineer y ColumnSelector definidas.


---
## 4. Construcción del Pipeline
Ensamblamos los pasos: Ingeniería -> Selección -> Escalado -> Modelo.

In [4]:
# Cargar lista de features seleccionadas
SELECTED_FEATURES_PATH = OUTPUT_PATH / "reports" / "07_selected_features_list.csv"
selected_features_df = pd.read_csv(SELECTED_FEATURES_PATH)
final_features_list = selected_features_df['Feature'].tolist()

print(f"✅ Features seleccionadas ({len(final_features_list)}): {final_features_list}")

# Definir el Preprocesador (Sin el modelo aún)
preprocessor = Pipeline([
    ('engineer', FeatureEngineer()), # 1. Crear variables (intensidad, ratios, dias)
    ('selector', ColumnSelector(columns=final_features_list)), # 2. Filtrar solo las 24 útiles
    ('scaler', StandardScaler()) # 3. Escalar (Mean=0, Std=1)
])

print("✅ Pipeline de preprocesamiento definido.")

✅ Features seleccionadas (24): ['score_riesgo', 'dias_activos_semanales', 'promedio_conexion', 'conexiones_mensuales', 'caracteristicas_usadas', 'dias_ultima_conexion', 'intensidad_uso', 'tickets_soporte', 'puntuacion_nps', 'tasa_crecimiento_uso', 'puntuacion_csat', 'ratio_carga_financiera', 'tasa_apertura_email', 'errores_pago', 'antiguedad', 'ingresos_totales', 'latitud', 'cargo_mensual', 'tiempo_resolucion', 'longitud', 'codigo_postal', 'edad', 'dias_desde_ultimo_contacto', 'tiempo_sesion_promedio']
✅ Pipeline de preprocesamiento definido.


---
## 5. Entrenamiento del Preprocesador
Ajustamos (`fit`) el preprocesador con los datos crudos (`data_clean.csv`). Esto calculará:
- La media y desviación estándar para el escalado.
- La fecha de referencia máxima para calcular antigüedades.

**Nota:** No re-entrenamos el modelo. Usamos el `09_best_model.pkl` ya optimizado.

In [5]:
print("⏳ Ajustando preprocesador con datos crudos...")

# Ajustar preprocesador
preprocessor.fit(X_raw)

print("✅ Preprocesador ajustado exitosamente.")

# Ensamblar el Pipeline Final (Preprocesador + Modelo)
# Nota: 'trained_model' ya está entrenado, así que el pipeline resultante estará listo para predecir.
full_pipeline = Pipeline(preprocessor.steps + [('model', trained_model)])

print("✅ Pipeline completo ensamblado.")

⏳ Ajustando preprocesador con datos crudos...
✅ Preprocesador ajustado exitosamente.
✅ Pipeline completo ensamblado.


---
## 6. Validación (Sanity Check)
Verificamos que el pipeline produzca las mismas predicciones que el flujo manual anterior.
Tomamos una muestra aleatoria, la pasamos por el pipeline y comparamos.

In [6]:
# Tomar una muestra de 5 registros crudos
sample_indices = [0, 100, 500, 1000, 5000]
X_sample = X_raw.iloc[sample_indices].copy()
y_true = y_raw.iloc[sample_indices]

# Predecir con el pipeline
y_pred_proba = full_pipeline.predict_proba(X_sample)[:, 1]
y_pred_class = full_pipeline.predict(X_sample)

print("🔍 Predicciones de prueba:")
for i, idx in enumerate(sample_indices):
    print(f"   Registro {idx}: Real={y_true.iloc[i]} -> Pred={y_pred_class[i]} (Prob: {y_pred_proba[i]:.4f})")

# Verificación técnica: El pipeline debe poder transformar sin errores
try:
    input_transformed = preprocessor.transform(X_sample)
    print(f"✅ Transformación exitosa. Shape intermedio: {input_transformed.shape} (Debe ser 5x24)")
except Exception as e:
    print(f"❌ Error en transformación: {e}")

🔍 Predicciones de prueba:
   Registro 0: Real=0 -> Pred=0 (Prob: 0.0586)
   Registro 100: Real=0 -> Pred=0 (Prob: 0.0405)
   Registro 500: Real=0 -> Pred=0 (Prob: 0.0427)
   Registro 1000: Real=0 -> Pred=1 (Prob: 0.6516)
   Registro 5000: Real=0 -> Pred=0 (Prob: 0.1166)
✅ Transformación exitosa. Shape intermedio: (5, 24) (Debe ser 5x24)


---
## 7. Exportación
Guardamos el pipeline listo para producción.

In [7]:
pipeline_filename = PIPELINE_PATH / "11_production_pipeline.pkl"

joblib.dump(full_pipeline, pipeline_filename)

print(f"💾 Pipeline exportado a: {pipeline_filename}")
print("🚀 Listo para despliegue.")

💾 Pipeline exportado a: ../../outputs/gian/pipelines/11_production_pipeline.pkl
🚀 Listo para despliegue.
